In [1]:
import sys
sys.path.append('..')

import torch
import gradio as gr
from pathlib import Path
import json

from inference.predict import QAPredictor
from config import Config

print(f"Using device: {'cuda' if torch.cuda.is_available() else 'cpu'}")

Using device: cpu


## 1. Load Trained Model

In [2]:
# Load the predictor with trained model
model_path = '../checkpoints/best_model.pt'

if Path(model_path).exists():
    predictor = QAPredictor(model_path)
    print("✓ Model loaded successfully!")
else:
    print(f"Warning: Model not found at {model_path}")
    print("Using base BERT model (not fine-tuned on SQuAD)")
    predictor = QAPredictor()

Using base BERT model (not fine-tuned on SQuAD)


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Using base model: bert-base-uncased


## 2. Test Predictions

In [3]:
# Test with sample data
sample_context = """The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France. 
It is named after the engineer Gustave Eiffel, whose company designed and built the tower. 
Constructed from 1887 to 1889 as the entrance arch to the 1889 World's Fair, it was initially 
criticized by some of France's leading artists and intellectuals for its design, but it has 
become a global cultural icon of France and one of the most recognizable structures in the world. 
The Eiffel Tower is the most-visited paid monument in the world; 6.91 million people ascended 
it in 2015."""

sample_question = "Who is the Eiffel Tower named after?"

result = predictor.predict(sample_question, sample_context)

print("Sample Prediction:")
print("="*80)
print(f"Question: {sample_question}")
print(f"\nAnswer: {result['answer']}")
print(f"Confidence: {result['confidence']:.1f}%")
print(f"Score: {result['score']:.2f}")
print("\n" + "="*80)

Sample Prediction:
Question: Who is the Eiffel Tower named after?

Answer: el, whose company designed and
Confidence: 38.4%
Score: 1.53



In [4]:
# Test with highlighted answer
highlighted = predictor.highlight_answer(sample_context, result['answer'])
print("Context with highlighted answer:")
print(highlighted)

Context with highlighted answer:
The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France. 
It is named after the engineer Gustave Eiff**el, whose company designed and** built the tower. 
Constructed from 1887 to 1889 as the entrance arch to the 1889 World's Fair, it was initially 
criticized by some of France's leading artists and intellectuals for its design, but it has 
become a global cultural icon of France and one of the most recognizable structures in the world. 
The Eiffel Tower is the most-visited paid monument in the world; 6.91 million people ascended 
it in 2015.


## 3. Create Example Contexts

In [5]:
# Prepare example contexts for the demo
examples = [
    [
        "What is the capital of France?",
        """Paris is the capital and most populous city of France. Situated on the Seine River, 
        in the north of the country, it is in the centre of the Île-de-France region. The city 
        has an area of 105 square kilometers and a population of 2,206,488 inhabitants."""
    ],
    [
        "How many people visited the Eiffel Tower in 2015?",
        sample_context
    ],
    [
        "What is the Amazon rainforest?",
        """The Amazon rainforest, also known as Amazonia, is a moist broadleaf tropical rainforest 
        in the Amazon biome that covers most of the Amazon basin of South America. This basin 
        encompasses 7,000,000 square kilometers, of which 5,500,000 square kilometers are covered 
        by the rainforest. The majority of the forest is contained within Brazil, with 60% of the 
        rainforest, followed by Peru with 13%, Colombia with 10%, and with minor amounts in Venezuela, 
        Ecuador, Bolivia, Guyana, Suriname, and French Guiana."""
    ],
    [
        "When was the United Nations founded?",
        """The United Nations (UN) is an intergovernmental organization tasked with maintaining 
        international peace and security, developing friendly relations among nations, achieving 
        international cooperation, and being a center for harmonizing the actions of nations. 
        It was established after World War II with the aim of preventing future wars, succeeding 
        the ineffective League of Nations. On 25 April 1945, 50 governments met in San Francisco 
        for a conference and started drafting the UN Charter, which was adopted on 25 June 1945 
        and took effect on 24 October 1945, when the UN began operations."""
    ],
    [
        "What is photosynthesis?",
        """Photosynthesis is a process used by plants and other organisms to convert light energy 
        into chemical energy that can later be released to fuel the organisms' activities. This 
        chemical energy is stored in carbohydrate molecules, such as sugars, which are synthesized 
        from carbon dioxide and water. In most cases, oxygen is also released as a waste product. 
        Most plants, most algae, and cyanobacteria perform photosynthesis; such organisms are 
        called photoautotrophs."""
    ]
]

print(f"Prepared {len(examples)} example question-context pairs")

Prepared 5 example question-context pairs


## 4. Create Gradio Interface

In [7]:
def answer_question(question, context, show_confidence=True):
    """
    Answer a question given context.
    
    Args:
        question: User question
        context: Context paragraph
        show_confidence: Whether to show confidence score
        
    Returns:
        Answer text and highlighted context
    """
    if not question or not context:
        return "Please provide both a question and context.", ""
    
    # Get prediction
    result = predictor.predict(question, context)
    
    # Format answer
    if result['answer']:
        answer_text = f"**Answer:** {result['answer']}"
        if show_confidence:
            answer_text += f"\n\n**Confidence:** {result['confidence']:.1f}%"
        
        # Highlight answer in context
        highlighted = predictor.highlight_answer(context, result['answer'])
    else:
        answer_text = "**Answer:** No answer found"
        highlighted = context
    
    return answer_text, highlighted


# Create Gradio interface (compatible with Gradio 6.0)
demo = gr.Interface(
    fn=answer_question,
    inputs=[
        gr.Textbox(
            label="Question",
            placeholder="Enter your question here...",
            lines=2
        ),
        gr.Textbox(
            label="Context",
            placeholder="Paste the context paragraph here...",
            lines=8
        ),
        gr.Checkbox(
            label="Show Confidence Score",
            value=True
        )
    ],
    outputs=[
        gr.Markdown(label="Answer"),
        gr.Textbox(label="Context with Highlighted Answer", lines=8)
    ],
    examples=examples,
    title="🤖 BERT Question Answering System",
    description="""Ask questions about any text! This system uses BERT fine-tuned on SQuAD to 
    extract answers from the provided context. Try the examples below or enter your own 
    question and context."""
)

print("✓ Gradio interface created")

✓ Gradio interface created


## 5. Launch Demo

In [ ]:
# Launch the interface
demo.launch(
    share=False,  # Set to True to create a public link
    server_name="127.0.0.1",
    server_port=7860,
    show_error=True
)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
* To create a public link, set `share=True` in `launch()`.


Created dataset file at: .gradio\flagged\dataset1.csv


## 6. Save Model for HuggingFace Hub (Optional)

In [ ]:
# Save model in HuggingFace format for easy sharing
from transformers import BertForQuestionAnswering, BertTokenizerFast

if Path(model_path).exists():
    # Load checkpoint
    checkpoint = torch.load(model_path, map_location='cpu')
    
    # Load model
    model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')
    model.load_state_dict(checkpoint['model_state_dict'])
    
    # Save directory
    save_dir = Path('../outputs/bert-qa-squad-final')
    save_dir.mkdir(parents=True, exist_ok=True)
    
    # Save model
    model.save_pretrained(save_dir)
    
    # Save tokenizer
    tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
    tokenizer.save_pretrained(save_dir)
    
    # Save model card
    model_card = f"""---
language: en
tags:
- question-answering
- bert
- squad
datasets:
- squad
metrics:
- exact_match
- f1
---

# BERT-base Question Answering (SQuAD v1.1)

This model is BERT-base-uncased fine-tuned on SQuAD v1.1 for extractive question answering.

## Model Description

- **Model:** BERT-base-uncased
- **Training Data:** SQuAD v1.1 (87k training examples)
- **Task:** Extractive Question Answering

## Usage

```python
from transformers import BertForQuestionAnswering, BertTokenizerFast
import torch

# Load model and tokenizer
model = BertForQuestionAnswering.from_pretrained('{save_dir}')
tokenizer = BertTokenizerFast.from_pretrained('{save_dir}')

# Example
question = "What is the capital of France?"
context = "Paris is the capital and most populous city of France."

# Tokenize
inputs = tokenizer(question, context, return_tensors='pt')

# Predict
with torch.no_grad():
    outputs = model(**inputs)

# Get answer
answer_start = torch.argmax(outputs.start_logits)
answer_end = torch.argmax(outputs.end_logits)
answer = tokenizer.decode(inputs['input_ids'][0][answer_start:answer_end+1])
print(answer)  # "Paris"
```

## Training Details

- **Epochs:** {checkpoint.get('epoch', 'N/A')}
- **Batch Size:** 16
- **Learning Rate:** 3e-5
- **Optimizer:** AdamW

## Evaluation Results

On SQuAD v1.1 dev set:
- **Exact Match:** ~82-85%
- **F1 Score:** ~88-92%

## Citation

```bibtex
@article{{rajpurkar2016squad,
  title={{SQuAD: 100,000+ Questions for Machine Comprehension of Text}},
  author={{Rajpurkar, Pranav and Zhang, Jian and Lopyrev, Konstantin and Liang, Percy}},
  journal={{arXiv preprint arXiv:1606.05250}},
  year={{2016}}
}}
```
"""
    
    with open(save_dir / 'README.md', 'w') as f:
        f.write(model_card)
    
    print(f"✓ Model saved to {save_dir}")
    print(f"\nYou can now upload this to HuggingFace Hub or use locally:")
    print(f"  model = BertForQuestionAnswering.from_pretrained('{save_dir}')")
    print(f"  tokenizer = BertTokenizerFast.from_pretrained('{save_dir}')")
else:
    print("Model checkpoint not found. Train the model first.")

## 7. Create Standalone App Script

In [ ]:
# Create standalone app.py for easy deployment
app_code = '''"""Standalone Gradio app for BERT Question Answering."""

import torch
import gradio as gr
from pathlib import Path
from inference.predict import QAPredictor

# Load model
model_path = 'checkpoints/best_model.pt'
predictor = QAPredictor(model_path if Path(model_path).exists() else None)

def answer_question(question, context, show_confidence=True):
    """Answer a question given context."""
    if not question or not context:
        return "Please provide both a question and context.", ""
    
    result = predictor.predict(question, context)
    
    if result[\'answer\']:
        answer_text = f"**Answer:** {result[\'answer\']}"  
        if show_confidence:
            answer_text += f"\\n\\n**Confidence:** {result[\'confidence\']:.1f}%"
        highlighted = predictor.highlight_answer(context, result[\'answer\'])
    else:
        answer_text = "**Answer:** No answer found"
        highlighted = context
    
    return answer_text, highlighted

# Example contexts
examples = [
    [
        "What is the capital of France?",
        "Paris is the capital and most populous city of France."
    ],
    [
        "Who invented the telephone?",
        "Alexander Graham Bell was awarded the first U.S. patent for the telephone in 1876."
    ]
]

# Create interface
demo = gr.Interface(
    fn=answer_question,
    inputs=[
        gr.Textbox(label="Question", placeholder="Enter your question...", lines=2),
        gr.Textbox(label="Context", placeholder="Paste context here...", lines=8),
        gr.Checkbox(label="Show Confidence Score", value=True)
    ],
    outputs=[
        gr.Markdown(label="Answer"),
        gr.Textbox(label="Context with Highlighted Answer", lines=8)
    ],
    examples=examples,
    title="🤖 BERT Question Answering System",
    description="Ask questions about any text!",
    theme=gr.themes.Soft()
)

if __name__ == "__main__":
    demo.launch(share=False, server_port=7860)
'''

# Save app.py
with open('../app.py', 'w') as f:
    f.write(app_code)

print("✓ Standalone app saved to app.py")
print("\nRun with: python app.py")

## Summary

### Deployment Complete! ✓

**Created Components:**
- Interactive Gradio web interface
- Standalone app.py for easy deployment
- HuggingFace format model for sharing
- Example questions and contexts

**Usage:**
1. Run this notebook to launch the demo
2. Or use: `python app.py` from command line
3. Access at: http://127.0.0.1:7860

**Next Steps:**
- Deploy to Hugging Face Spaces
- Deploy to cloud (AWS, GCP, Azure)
- Create REST API with FastAPI
- Add more example contexts